In [5]:
!pip install vectorbt
!pip install condacolab
import condacolab
condacolab.install()
!conda install -c conda-forge ta-lib
!pip install vectorbt

  Using cached condacolab-0.1.9-py3-none-any.whl.metadata (5.6 kB)
Using cached condacolab-0.1.9-py3-none-any.whl (7.2 kB)
✨🍰✨ Everything looks OK!
Channels:
 - conda-forge
Platform: linux-64
Solving environment: | / - \ done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.7.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.



In [1]:
import vectorbt as vbt
import talib as ta
import requests
import json
import math
import pandas_datareader as web
import numpy as np
import pandas as pd
import time
import datetime



In [3]:
class GetBinanceData:
    @staticmethod
    def get_data(symbol, interval, startDate=datetime.datetime(2020, 1, 1), endDate=datetime.datetime.now()):
        dataList = []
        current_date = datetime.datetime.now()
        start_date = startDate

        delta_units = GetBinanceData.convert_to_minutes(interval)
        print(delta_units)
        delta = datetime.timedelta(minutes=delta_units)

        while start_date < current_date:
            end_date = start_date + delta
            if end_date > current_date:
                end_date = current_date
            # Convert datetime objects to milliseconds
            start_ms = int(start_date.timestamp() * 1000)
            end_ms = int(end_date.timestamp() * 1000)

            start_str = start_date.strftime('%Y-%m-%d %H:%M:%S')
            end_str = end_date.strftime('%Y-%m-%d %H:%M:%S')

            print(f"Fetching data from {start_str} to {end_str}")
            data = GetBinanceData.get_historical_data(symbol, interval, start_ms, end_ms)
            if data:
                dataList.extend(data)
            start_date = end_date
            time.sleep(1)

        return dataList

    @staticmethod
    def get_historical_data(symbol, interval, start_time, end_time):
        url = f"https://api.binance.us/api/v3/klines"
        params = {
            'symbol': symbol.upper(),
            'interval': interval,
            'startTime': start_time,
            'endTime': end_time,
            'limit': 1000
        }
        print(params)
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            return data
        else:

            print(f"Failed to fetch data. Status code: {response.json()}")
            return None

    @staticmethod
    def convert_to_minutes(time_str):
        time_mapping = {
            'm': 1,
            'h': 60,
            'd': 1440,
            'w': 10080,
            'M': 43200
        }

        num = int(time_str[:-1])
        unit = time_str[-1]

        return (num * time_mapping[unit])*1000

    def data_to_dataframe(data):
      # Define the column names based on Binance API response
      columns = [
          "timestamp", "open", "high", "low", "close", "volume",
          "Close Time", "Quote Asset Volume", "Number of Trades",
          "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore"
      ]
      # Convert the data to a pandas DataFrame
      df = pd.DataFrame(data, columns=columns)
      # Convert timestamp columns to datetime
      df["timestamp"] = pd.to_datetime(df['timestamp'])
      df["Close Time"] = pd.to_datetime(df["Close Time"], unit='ms')
      # Convert numeric columns to float
      numeric_columns = ["open", "high", "low", "close", "volume", "Quote Asset Volume",
                        "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume"]
      df[numeric_columns] = df[numeric_columns].astype(float)
      return df

    @staticmethod
    def get_data_dataFrame(symbol, interval, startDate=datetime.datetime(2020, 1, 1), endDate=datetime.datetime.now()):
        data = GetBinanceData.get_data(symbol, interval, startDate, endDate)
        return GetBinanceData.data_to_dataframe(data)

In [4]:
timeperiod = '5m'

symbol = "adausdt"
data = GetBinanceData.get_data(symbol, timeperiod, startDate=datetime.datetime(2022, 1, 1), endDate=datetime.datetime.now())


5000
Fetching data from 2022-01-01 00:00:00 to 2022-01-04 11:20:00
{'symbol': 'ADAUSDT', 'interval': '5m', 'startTime': 1640995200000, 'endTime': 1641295200000, 'limit': 1000}
Fetching data from 2022-01-04 11:20:00 to 2022-01-07 22:40:00
{'symbol': 'ADAUSDT', 'interval': '5m', 'startTime': 1641295200000, 'endTime': 1641595200000, 'limit': 1000}
Fetching data from 2022-01-07 22:40:00 to 2022-01-11 10:00:00
{'symbol': 'ADAUSDT', 'interval': '5m', 'startTime': 1641595200000, 'endTime': 1641895200000, 'limit': 1000}
Fetching data from 2022-01-11 10:00:00 to 2022-01-14 21:20:00
{'symbol': 'ADAUSDT', 'interval': '5m', 'startTime': 1641895200000, 'endTime': 1642195200000, 'limit': 1000}
Fetching data from 2022-01-14 21:20:00 to 2022-01-18 08:40:00
{'symbol': 'ADAUSDT', 'interval': '5m', 'startTime': 1642195200000, 'endTime': 1642495200000, 'limit': 1000}
Fetching data from 2022-01-18 08:40:00 to 2022-01-21 20:00:00
{'symbol': 'ADAUSDT', 'interval': '5m', 'startTime': 1642495200000, 'endTime':